In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep

In [5]:
data = []
fetch_images = False

user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.67",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:89.0) Gecko/20100101 Firefox/89.0",
    "Mozilla/5.0 (Linux; Android 10; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Mobile Safari/537.36",
    "Mozilla/5.0 (Android 10; Mobile; rv:89.0) Gecko/89.0 Firefox/89.0",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (iPad; CPU OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Mobile/15E148 Safari/604.1"
]


for i in range(10):
    error = True
    # sleep(2)
    # print(i)
    # Define the URL to scrape
    base_url = "https://www.vivareal.com.br"
    url = base_url + f"/venda/santa-catarina/florianopolis/?pagina={i + 1}#onde=,Santa%20Catarina,Florian%C3%B3polis,,,,,city,BR%3ESanta%20Catarina%3ENULL%3EFlorianopolis,-27.594808,-48.556915,&itl_id=1000183&itl_name=vivareal_-_botao-cta_buscar_to_vivareal_resultado-pesquisa"

    while error:
        user_agent_c = 0
        # Set the desired user agent
        headers = {
            "User-Agent": user_agents[user_agent_c]
        }
        
        try:
            # Send an HTTP request with the specified user agent
            response = requests.get(url, headers=headers)
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Find the relevant table containing property data
            table = soup.find("div", attrs={"class": "results-list js-results-list"})
            
            rows = table.find_all("div", class_="js-card-selector")
            
            for row in rows:
                info = row.find
            
                anuncio_url = (
                    base_url +
                    row.find(class_="property-card__labels-container js-main-info js-listing-labels-link").get("href")
                )

                # Image data is not working because the website throttle the crawler and prevent more ingestions
                
                # if fetch_images:
                #     anuncio_resp = requests.get(anuncio_url, headers=headers)
                #     anuncio_soup = BeautifulSoup(anuncio_resp.content, "html.parser")
                
                #     images_ul = anuncio_soup.find(class_="carousel__container js-carousel-scroll")
                #     images_data = []
                    
                #     for image in images_ul:
                #         image_tag = image.find("img")
                #         if not isinstance(image_tag, int):
                #             images_data.append({
                #                 "alt": image_tag.get("alt"),
                #                 "src": image_tag.get("src")
                #             })
                
                data.append({
                    "title": row.find(class_="property-card__title js-cardLink js-card-title").text.strip(),
                    
                    # "imagens": images_data if fetch_images else [],
                    
                    "url": anuncio_url,
                    
                    "endereco": row.find(class_="property-card__address").text.strip(),
                    
                    "area": row.find(class_="property-card__detail-item property-card__detail-area").find(class_="property-card__detail-value js-property-card-value property-card__detail-area js-property-card-detail-area").text.strip(),
                    "area_label": row.find(class_="property-card__detail-item property-card__detail-area").find(class_="property-card__detail-text js-property-card-detail-text").text.strip(),
                    
                    "quartos": row.find(class_="property-card__detail-item property-card__detail-room js-property-detail-rooms").find(class_="property-card__detail-value js-property-card-value").text.strip(),
                    "quartos_label": row.find(class_="property-card__detail-item property-card__detail-room js-property-detail-rooms").find(class_="property-card__detail-text js-property-card-detail-text").text.strip(),
                    
                    "banheiros": row.find(class_="property-card__detail-item property-card__detail-bathroom js-property-detail-bathroom").find(class_="property-card__detail-value js-property-card-value").text.strip(),
                    "banheiros_label": row.find(class_="property-card__detail-item property-card__detail-bathroom js-property-detail-bathroom").find(class_="property-card__detail-text js-property-card-detail-text").text.strip(),
                    
                    "vagas": row.find(class_="property-card__detail-item property-card__detail-garage js-property-detail-garages").find(class_="property-card__detail-value js-property-card-value").text.strip(),
                    "vagas_label": row.find(class_="property-card__detail-item property-card__detail-garage js-property-detail-garages").find(class_="property-card__detail-text js-property-card-detail-text").text.strip(),
                    
                    "preco": row.find(class_="property-card__price js-property-card-prices js-property-card__price-small").p.text.strip(),
                    
                    "fonte": url
                })

                error = False
        except Exception as e:
            error = True
            print(f"Error ingesting url: {anuncio_url}, Error: {e}")
            if user_agent_c < len(user_agents) - 1:
                user_agent_c += 1
                headers = {
                    "User-Agent": user_agents[user_agent_c]
                }
            else:
                break
            
            sleep(10)

In [6]:
data

[{'title': 'Apartamento com 2 Quartos à Venda, 77m²',
  'url': 'https://www.vivareal.com.br/imovel/apartamento-2-quartos-ingleses-do-rio-vermelho-bairros-florianopolis-com-garagem-77m2-venda-RS589000-id-2722640377/',
  'endereco': 'Rua das Gaivotas, 230 - Ingleses do Rio Vermelho, Florianópolis - SC',
  'area': '77',
  'area_label': 'm²',
  'quartos': '2',
  'quartos_label': 'Quartos',
  'banheiros': '1',
  'banheiros_label': 'Banheiro',
  'vagas': '1',
  'vagas_label': 'Vaga',
  'preco': 'R$ 589.000',
  'fonte': 'https://www.vivareal.com.br/venda/santa-catarina/florianopolis/?pagina=1#onde=,Santa%20Catarina,Florian%C3%B3polis,,,,,city,BR%3ESanta%20Catarina%3ENULL%3EFlorianopolis,-27.594808,-48.556915,&itl_id=1000183&itl_name=vivareal_-_botao-cta_buscar_to_vivareal_resultado-pesquisa'},
 {'title': 'RESIDENCIAL BACK GREEN TOWERS - 73m²',
  'url': 'https://www.vivareal.com.br/imoveis-lancamento/residencial-back-green-towers-73m-2569361604/',
  'endereco': 'Rua Trajano Margarida, 180 - Tr

In [7]:
df = pd.DataFrame(data)

In [8]:
df

,title,url,endereco,area,area_label,quartos,quartos_label,banheiros,banheiros_label,vagas,vagas_label,preco,fonte
0,"Apartamento com 2 Quartos à Venda, 77m²",https://www.vivareal.com.br/imovel/apartamento...,"Rua das Gaivotas, 230 - Ingleses do Rio Vermel...",77,m²,2,Quartos,1,Banheiro,1,Vaga,R$ 589.000,https://www.vivareal.com.br/venda/santa-catari...
1,RESIDENCIAL BACK GREEN TOWERS - 73m²,https://www.vivareal.com.br/imoveis-lancamento...,"Rua Trajano Margarida, 180 - Trindade, Florian...",73,m²,2,Quartos,1,Banheiro,1,Vaga,R$ 998.000,https://www.vivareal.com.br/venda/santa-catari...
2,"Apartamento com 2 Quartos à Venda, 72m²",https://www.vivareal.com.br/imovel/apartamento...,"Rua Jornalista Manoel Menezes, 149 - Itacorubi...",72,m²,2,Quartos,2,Banheiros,2,Vagas,R$ 890.000,https://www.vivareal.com.br/venda/santa-catari...
3,"Apartamento com 3 Quartos à Venda, 100m²",https://www.vivareal.com.br/imovel/apartamento...,"Rua Pirineus, 90 - Córrego Grande, Florianópol...",100,m²,3,Quartos,3,Banheiros,2,Vagas,R$ 670.000 Preço abaixo do mercado,https://www.vivareal.com.br/venda/santa-catari...
4,"Casa com 2 Quartos à Venda, 109m²",https://www.vivareal.com.br/imovel/sobrado-2-q...,"Servidão Luiz Duarte Soares, 500 - São João do...",109,m²,2,Quartos,3,Banheiros,4,Vagas,R$ 580.000,https://www.vivareal.com.br/venda/santa-catari...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,"Apartamento com 2 Quartos à Venda, 75m²",https://www.vivareal.com.br/imovel/apartamento...,Avenida Luiz Boiteux Piazza - Cachoeira do Bom...,75,m²,2,Quartos,2,Banheiros,1,Vaga,R$ 978.770,https://www.vivareal.com.br/venda/santa-catari...
356,"Sala/Conjunto à Venda, 220m²",https://www.vivareal.com.br/imovel/sala-comerc...,"Rua Presidente Nereu Ramos, 19 - Centro, Flori...",220,m²,--,Quarto,--,Banheiro,2,Vagas,R$ 2.500.000,https://www.vivareal.com.br/venda/santa-catari...
357,"Apartamento com Quarto à Venda, 105m²",https://www.vivareal.com.br/imovel/apartamento...,Rodovia Tertuliano Brito Xavier - Canasvieiras...,105,m²,1,Quarto,1,Banheiro,1,Vaga,R$ 1.750.000,https://www.vivareal.com.br/venda/santa-catari...
358,"Casa com 5 Quartos à Venda, 230m²",https://www.vivareal.com.br/imovel/casa-5-quar...,"Rua da Lua Cheia, 100 - Lagoa do Peri, Florian...",230,m²,5,Quartos,1,Banheiro,2,Vagas,R$ 1.263.000,https://www.vivareal.com.br/venda/santa-catari...


In [9]:
# df.to_json("2024-06-20_vivareal_3krow.json", orient="records")